In [1]:
import h5py
import numpy as np 
with h5py.File('images_training.h5','r') as H:
    data = np.copy(H['data']) 
with h5py.File('labels_training.h5','r') as H:
    label = np.copy(H['label'])

In [2]:
data.shape

(30000, 28, 28)

In [3]:
label[-1]

2

In [4]:
m=data.shape[1]*data.shape[2]
data1 = data.reshape(30000,m)
del data

In [5]:
data1.shape

(30000, 784)

In [6]:
target = set()
for i in label:
    target.add(i)
d = len(list(target))
d

10

In [7]:
# ovr, label "1" vs. not label "1" 
label1 = np.array([])
for i in range(d):
    label_tmp=np.copy(label)
    for tmp in range(len(label_tmp)):
        if label_tmp[tmp] != i:
            label_tmp[tmp] = 0
        else:
            label_tmp[tmp] = 1
    label_tmp= label_tmp.reshape(-1,1)
    if i == 0:
        label1 = np.copy(label_tmp)
    else:
        label1 = np.append(label1, label_tmp, axis=1)
del label_tmp

In [8]:
label1.shape


(30000, 10)

In [9]:
label[-1]

2

In [10]:
label1

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=uint8)

In [11]:
#used scikit-learn library, need to implemenet ourselve later
from sklearn.preprocessing import normalize
data_normalize = normalize(data1)
del data1

In [12]:
data_normalize.shape

(30000, 784)

In [13]:
#from tutorial
def logistic_loss_and_grad(weights, inputs, targets):

    #derivative of the loss function
    z = np.dot(inputs, weights)
    exp_z = np.exp(z)
    temp = -exp_z/(1 + exp_z) + targets
#     print(temp.shape, inputs.shape, (temp*inputs).shape)
    dloss = -np.sum(temp*inputs, axis=0)[:, np.newaxis]

    return dloss

In [17]:
import numpy.linalg as lng
# weights = np.random.random(data1.shape[1])[:, np.newaxis]
weights = np.array([])
eta = 0.03
# n_iter = 2000
n_iter = 2000
eps = 1e-10

for i in range(d):
    weight = np.random.random(data_normalize.shape[1])[:, np.newaxis]
    current_label = (label1[:,i]).reshape(-1,1)
    print(data_normalize.shape)
    for epoch in range(n_iter):
        dloss = logistic_loss_and_grad(weight, data_normalize, current_label)
        weight_new = weight - dloss * eta
        diff = lng.norm(weight_new - weight)
        weight = weight_new
        if diff < eps:
            break
    if i == 0:
        weights = weight
    else:
        weights = np.append(weights, weight, axis = 1)

(30000, 784)
(30000, 784)
(30000, 784)
(30000, 784)
(30000, 784)
(30000, 784)
(30000, 784)
(30000, 784)
(30000, 784)
(30000, 784)


In [18]:
weights.shape

(784, 10)

In [19]:

with h5py.File('images_testing.h5','r') as H:
    data_testing = np.copy(H['data']) 
with h5py.File('labels_testing_2000.h5','r') as H:
    label_testing = np.copy(H['label'])

In [20]:
data_testing = np.delete(data_testing, np.s_[2000:5000], 0)

In [21]:
data_testing = data_testing.reshape(2000,-1)
print(data_testing.shape)
print(label_testing.shape)

(2000, 784)
(2000,)


In [22]:
calculation = data_testing.dot(weights)
calculation

array([[ -26588.64537108,  -34497.55284898, -102013.6067531 , ...,
        -214187.88934589, -120094.7800282 , -149605.45661504],
       [-185567.1784891 ,  -41833.06576639,   38126.71819234, ...,
        -553977.37693457, -185231.72156837, -502443.15011693],
       [-289205.26934906,  -41102.25433803,  -93281.69258373, ...,
        -541322.74224855,  -95478.65061925, -492076.22259669],
       ...,
       [-219720.17584783,  -25965.94184883,   88796.907172  , ...,
        -511842.41539672, -145077.58102217, -444442.58728756],
       [-102855.55034695, -113198.14669626, -235869.1326902 , ...,
        -170873.61297186, -219234.20334562,   22801.66980436],
       [ -13911.25216627,  -15928.43617732,  -22364.20819078, ...,
        -142546.74090202,  -34005.73262734,  -86669.98632836]])

In [23]:
result = np.argmax(calculation, axis=1)
result.shape

(2000,)

In [24]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion_matrix(label_testing, result)

array([[136,   1,   2,  12,   0,   0,  25,   0,   2,   0],
       [  0, 183,   3,   5,   0,   0,   0,   0,   0,   0],
       [  2,   1, 185,   3,   3,   1,  13,   0,   2,   0],
       [  4,   3,   4, 176,   0,   0,   4,   0,   0,   0],
       [  0,  12,  84,  19,  34,   0,  61,   0,   2,   0],
       [  1,   0,   0,   2,   0, 186,   1,   8,   2,  14],
       [ 21,   6,  39,  11,   4,   0, 116,   0,   3,   0],
       [  0,   0,   0,   0,   0,  10,   0, 182,   0,   6],
       [  0,   0,   2,   2,   1,   2,   7,   2, 203,   0],
       [  0,   0,   1,   0,   0,   3,   1,   5,   0, 177]], dtype=int64)

In [26]:
print(accuracy_score(label_testing, result)*100, "%")

78.9 %
